In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from collections import OrderedDict
from datetime import datetime
# also requires:
# `pip install tables`
# to be able to write to hdf5 files

In [2]:
BASE_URL = 'http://www.basketball-reference.com/'
TEAM = 'MIN'
YR = 2017 # second year in 2016/2017
soup = BeautifulSoup(requests.get('http://www.basketball-reference.com/teams/%s/%d_games.html' % (TEAM, YR)).text,'lxml')

In [3]:
# extract box score link
box_scores = soup.find('table',{'id':'games'}).findAll('td',{'data-stat':'box_score_text'})
links = [box_scores[ii].a['href'] for ii in range(len(box_scores))]
print links[0]

/boxscores/201610260MEM.html


In [4]:
# create play by play link
def _get_pbp_link(link):
    sp = link.split('/')
    newlink = sp[-2] + '/pbp/' + sp[-1]
    return newlink
def _get_shot_chart_link(link):
    sp = link.split('/')
    newlink = sp[-2] + '/shot-chart/' + sp[-1]
    return newlink
pbp_links = [_get_pbp_link(link) for link in links]
sc_links = [_get_shot_chart_link(link) for link in links]
bs_links = [BASE_URL+link for link in links]
print BASE_URL + pbp_links[0]
print BASE_URL + sc_links[0]
print bs_links[0]

http://www.basketball-reference.com/boxscores/pbp/201610260MEM.html
http://www.basketball-reference.com/boxscores/shot-chart/201610260MEM.html
http://www.basketball-reference.com//boxscores/201610260MEM.html


In [5]:
# extract table from pbp page
soup_pbp = BeautifulSoup(requests.get(BASE_URL+pbp_links[0]).text,'lxml')

table = soup_pbp.find('table',{'id':'pbp'})
rows =table.findAll('tr')
data = {}
data['times'] = []
data['ht_plays'] = []
data['at_plays'] = []
data['score_after'] = []
data['ht_pts'] = []
data['at_pts'] = []
data['qrtr'] = []
qrtr = 0
for row in rows:
    dat = row.findAll('td')
    if len(dat)== 2 and dat[1].text[:5]=='Start':
        qrtr += 1
    if len(dat)==6:
        data['times'].append(dat[0].text)
        data['ht_plays'].append(dat[1].text)
        data['ht_pts'].append(dat[2].text)
        data['score_after'].append(dat[3].text)
        data['at_pts'].append(dat[4].text)
        data['at_plays'].append(dat[5].text)
        data['qrtr'].append(qrtr)
        
pbpFrame = pd.DataFrame(data)

In [6]:
soup_sc = BeautifulSoup(requests.get(BASE_URL+sc_links[0]).text,'lxml')

def _extract_shot_chart(soup_sc, team=None):
    id_tag = 'shots-' + team.upper()
    court_width=50
    court_len=47
    shot_dict = {}
    shot_dict['shot_xs'] = []
    shot_dict['shot_ys'] = []
    shot_dict['shot_time'] = []
    shot_dict['player'] = []
    shots = soup_sc.find('div',{'id':id_tag}).findAll('div',{'class','tooltip'})
    for shot in shots:
        sp = shot['tip'].split('<br>')
        shot_dict['player'].append(shot['class'][2][2:])
        shot_dict['shot_time'].append(sp[0].split(',')[1][1:-10])
        # get shot coordinates. Image is 500 x 472 px,
        # court is 50 ft. x 47 ft, so just divide by 10.
        shot_dict['shot_xs'].append(int(shot['style'].split(':')[-1][:-3])/10.)
        shot_dict['shot_ys'].append(int(shot['style'].split(':')[1][:-7])/10.)

    shotFrame = pd.DataFrame(shot_dict)
    return shotFrame
shotFrame = _extract_shot_chart(soup_sc, team='mem')

In [7]:
shotFrame.to_hdf('test_game.hdf5','shotFrame')
pbpFrame.to_hdf('test_game.hdf5','pbpFrame')

/Users/meyers/opt/bball/lib/python2.7/site-packages/pandas/core/generic.py:1138: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->unicode,key->block1_values] [items->['at_plays', 'at_pts', 'ht_plays', 'ht_pts', 'score_after', 'times']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [8]:
soup_bs = BeautifulSoup(requests.get(bs_links[0]).text,'lxml')
print bs_links[0]
def extract_box_score(soup_bs, team=None):
    id_tag = 'box_' + team.lower() + '_basic'
    tab = soup_bs.find('table',{'id':id_tag})
    if len(tab)==0:
        raise ValueError('You entered team: %s, which does not have a box score on this page' % team)
    header_strs = [td.text for td in tab.findAll('tr')[1].findAll('th')]
    box_score = OrderedDict()
    for h in header_strs:
        box_score[h] = []
    for ii,tr in enumerate(tab.findAll('tr')):
        tds = [td.text for td in tr.findAll('td')]
        # if we're at the last row...continue
        # if we have more than one non-header column
        if len(tds) > 1:
            # loop through header strings
            for jj, head in enumerate(header_strs):
                if jj == 0:
                    # if we're on the first header string, then it's the player's name
                    # and that's in a <th> tag, not <td> tag. If it's the last row,
                    # then the player's "name" ("Team Totals") is in a different type
                    # of th tag.
                    if ii == len(tab.findAll('tr')) - 1:
                        box_score[head].append(tr.find('th').text)
                    else:
                        box_score[head].append(tr.find('th')['data-append-csv'])
                    continue
                else:
                    # otherwise, we loop through tds, and since
                    # these are indexed off-set from headers (since
                    # player names start), we index with jj-1
                    if jj ==1:
                        box_score[head].append(tds[jj-1])
                    else:
                        if tds[jj-1]=='':
                            tds[jj-1] = 0
                        box_score[head].append(float(tds[jj-1]))
    box_score = pd.DataFrame(box_score)
    return box_score

away_bs = extract_box_score(soup_bs, team='min')
home_bs = extract_box_score(soup_bs, team='mem')
print away_bs
print home_bs


http://www.basketball-reference.com//boxscores/201610260MEM.html
       Starters     MP    FG   FGA    FG%   3P   3PA    3P%    FT   FTA  ...   \
0     townska01  36:29   9.0  20.0  0.450  2.0   5.0  0.400   1.0   1.0  ...    
1     wiggian01  34:29   7.0  14.0  0.500  1.0   1.0  1.000  10.0  14.0  ...    
2     dienggo01  34:01   5.0  10.0  0.500  0.0   1.0  0.000   2.0   2.0  ...    
3     lavinza01  33:50   6.0  14.0  0.429  3.0   6.0  0.500   4.0   5.0  ...    
4     rubiori01  33:27   1.0   5.0  0.200  0.0   1.0  0.000   2.0   5.0  ...    
5     bjeline01  15:23   1.0   4.0  0.250  0.0   1.0  0.000   1.0   2.0  ...    
6      dunnkr01  15:13   4.0   6.0  0.667  0.0   1.0  0.000   0.0   0.0  ...    
7      rushbr01  13:39   0.0   3.0  0.000  0.0   1.0  0.000   0.0   0.0  ...    
8     muhamsh01  13:31   1.0   6.0  0.167  0.0   1.0  0.000   0.0   0.0  ...    
9     aldrico01   9:58   2.0   4.0  0.500  0.0   0.0  0.000   0.0   0.0  ...    
10  Team Totals    240  36.0  86.0  0.419  6

In [9]:
def _get_team(soup_bs,team='home'):
    score_div = soup_bs.find('div',{'class':'scorebox'})
    if team == 'home':
        t = score_div.findAll('strong')[1].a['href'].split('/')[2].lower()
    elif team=='away':
        t = score_div.findAll('strong')[0].a['href'].split('/')[2].lower()
    else:
        raise ValueError('team must be either "home" or "away". Default is "home"')
    return t
print _get_team(soup_bs, team='home')
print _get_team(soup_bs, team='away')

mem
min


In [10]:
def _get_date(soup_bs):
    content_div = soup_bs.find('div',{'id':'content'})
    month_day = content_div.find('h1').text.split(',')[-2]
    year = content_div.find('h1').text.split(',')[-1]
    date = datetime.strptime(month_day.strip() + year,'%B %d %Y')
    return date
date = _get_date(soup_bs)
print date

2016-10-26 00:00:00


In [23]:
?pd.DataFrame()

In [70]:
import numpy as np
def _get_schedule(soup):
    tab = soup.find('table',{'id':'games'})
    list_vals =pd.read_html('<table>' + str(tab.find('tbody'))+'</table>')
    headers = []
    sched = list_vals[0]
    for ii,th in enumerate(tab.find('thead').findAll('th')):
            headers.append(th['data-stat'])
    print headers
    sched.columns = headers
    # fix game locations
    print type(sched['game_location'])
    away = np.where(sched['game_location']=='@')
    sched['game_location'] = sched['game_location'].replace('@','A')
    sched['game_location'] = sched['game_location'].replace(np.nan,'H')
    sched['overtimes'] = sched['overtimes'].replace(np.nan,0)
    sched = sched.replace(np.nan,'--')
    return sched
print _get_schedule(soup)


['g', 'date_game', 'game_start_time', 'network', 'box_score_text', 'game_location', 'opp_name', 'game_result', 'overtimes', 'pts', 'opp_pts', 'wins', 'losses', 'game_streak', 'game_remarks']
<class 'pandas.core.series.Series'>
     g          date_game game_start_time network box_score_text  \
0    1  Wed, Oct 26, 2016        8:00p ET      --      Box Score   
1    2  Sat, Oct 29, 2016       10:30p ET      --      Box Score   
2    3   Tue, Nov 1, 2016        8:00p ET      --      Box Score   
3    4   Thu, Nov 3, 2016        8:00p ET      --      Box Score   
4    5   Sat, Nov 5, 2016        6:00p ET      --      Box Score   
5    6   Tue, Nov 8, 2016        7:30p ET      --      Box Score   
6    7   Wed, Nov 9, 2016        7:00p ET      --      Box Score   
7    8  Sat, Nov 12, 2016        8:00p ET      --      Box Score   
8    9  Sun, Nov 13, 2016        7:00p ET      --      Box Score   
9   10  Tue, Nov 15, 2016        8:00p ET      --      Box Score   
10  11  Thu, Nov 17, 2016